In [10]:
import json
from pathlib import Path
import subprocess
import os

In [72]:
def write_tsp_file(points, filename="problem.tsp"):
    with open(filename, "w") as f:
        f.write("NAME : problem\n")
        f.write("TYPE : TSP\n")
        f.write("DIMENSION : {}\n".format(len(points)))
        f.write("EDGE_WEIGHT_TYPE : EUC_2D\n")
        f.write("NODE_COORD_SECTION\n")
        for i, (x, y) in enumerate(points):
            f.write(f"{i+1} {x*100} {y*100}\n")
        f.write("EOF\n")


def run_concorde(tsp_filename: Path, sol_filename: Path):
    result = subprocess.run([
        "docker", "run", "--rm", "-t",
        "-v", r"F:\admin\PycharmProjects\CG\Competition\optimisation\cg_tsp - HARDCODED\tsplib:/usr/local/opt/concorde",
        "alehkot/concorde-tsp:1.0",
        tsp_filename.name
    ])
    print(result)
    if result.returncode != 0:
        print("Error:", result.stdout)

def text_to_points(text):
    lines = text.strip().split("\n")
    points = []
    for line in lines[1:]:
        if line.strip():
            x, y = map(int, line.split())
            points.append((x, y))
    return points

def prepare(input_filename: Path, tsp_filename: Path):
    with open(input_filename, 'r') as f:
        data = json.load(f)

    print(f"\nTest: {data['isTest']} \tValidator: {data['isValidator']}")

    write_tsp_file(text_to_points(data["testIn"]), tsp_filename)


def get_hash(input_filename: Path) -> str:
    with open(input_filename, 'r') as f:
        data = json.load(f)

    all_pts = []
    for line in data["testIn"].split("\n"):
        if line.strip():
            all_pts.extend([int(x) for x in line.split()])
    return str(hash(tuple(all_pts)))


def load_solution(sol_filename: Path) -> str:
    with open(sol_filename, 'r') as f:
        lines = f.readlines()
        tour = "".join(lines[1:]).replace("\n", "") + "0"
    return tour

In [73]:
source_folder = Path('testcases/')
target_folder = Path('tsplib/')
target_folder.mkdir(exist_ok=True)

ans = {}

for input_filename in source_folder.glob('*.json'):

    tsp_filename = target_folder / (input_filename.stem + ".tsp")
    sol_filename = target_folder / (input_filename.stem + ".sol")

    code = get_hash(input_filename)

    prepare(input_filename, tsp_filename)

    run_concorde(tsp_filename, sol_filename)

    solution = load_solution(sol_filename)

    ans[code] = solution
    
    # break
with open("solution.json", "w") as f:
    json.dump(ans, f)


Test: true 	Validator: false
CompletedProcess(args=['docker', 'run', '--rm', '-t', '-v', 'F:\\admin\\PycharmProjects\\CG\\Competition\\optimisation\\cg_tsp - HARDCODED\\tsplib:/usr/local/opt/concorde', 'alehkot/concorde-tsp:1.0', 'test1.tsp'], returncode=255)
Error: None

Test: false 	Validator: true
CompletedProcess(args=['docker', 'run', '--rm', '-t', '-v', 'F:\\admin\\PycharmProjects\\CG\\Competition\\optimisation\\cg_tsp - HARDCODED\\tsplib:/usr/local/opt/concorde', 'alehkot/concorde-tsp:1.0', 'test10.tsp'], returncode=255)
Error: None

Test: false 	Validator: true
CompletedProcess(args=['docker', 'run', '--rm', '-t', '-v', 'F:\\admin\\PycharmProjects\\CG\\Competition\\optimisation\\cg_tsp - HARDCODED\\tsplib:/usr/local/opt/concorde', 'alehkot/concorde-tsp:1.0', 'test11.tsp'], returncode=255)
Error: None

Test: false 	Validator: true
CompletedProcess(args=['docker', 'run', '--rm', '-t', '-v', 'F:\\admin\\PycharmProjects\\CG\\Competition\\optimisation\\cg_tsp - HARDCODED\\tsplib:/u